<p>Измалкова Дарья БКЛ182</p>

<h1>Домашнее задание №2 часть 1</h1>

In [1]:
from pymystem3 import Mystem
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from pymorphy2 import MorphAnalyzer
from collections import Counter
import json
import re
import os


morph = MorphAnalyzer()
m = Mystem()

<h3>Для обработки взята книга Ф.М. Достоевского "Игрок"</h3>

In [2]:
with open('igrok.txt', encoding='utf-8') as f:
    text = f.read()

<h3>Парсинг с помощью Mystem</h3>
<p>Питоновский модуль работал слишком медленно (больше нескольких часов), поэтому я воспользовалась консольной версией</p>
<p>Опции "-gin" - печатать грамматическую информацию, склеивать информацию словоформ при одной лемме, каждое слово с новой строки</p>

In [3]:
%%time
import os

inp = "igrok.txt"
outp = "igrok.json"
mystem_path = os.path.join('/Users/User', 'mystem.exe')

input_filename = os.path.abspath(inp)
output_filename = os.path.abspath(outp)
os.system(f"{mystem_path} {'-gin'} {'--format json'} {input_filename} {output_filename}") 

Wall time: 2.23 s


0

<p>Проверка, что файл нормально обработался и записался</p>

In [4]:
ana_list = []
with open('igrok.json', encoding='utf-8') as f:
    text2 = f.read()
    lines = text2.splitlines()

for line in lines:
    data = json.loads(line)
    ana_list.append(data)

print(ana_list[100:105])

[{'analysis': [{'lex': 'сестра', 'gr': 'S,жен,од=(пр,ед|дат,ед)'}], 'text': 'сестре'}, {'analysis': [{'lex': 'быть', 'gr': 'V,нп=прош,ед,изъяв,сред,несов'}, {'lex': 'было', 'gr': 'PART='}], 'text': 'Было'}, {'analysis': [{'lex': 'ясно', 'gr': 'ADV='}, {'lex': 'ясный', 'gr': 'A=ед,кр,сред'}], 'text': 'ясно'}, {'analysis': [{'lex': 'что', 'gr': 'CONJ='}, {'lex': 'что', 'gr': 'SPRO,ед,сред,неод=(вин|им)'}, {'lex': 'что', 'gr': 'ADVPRO='}], 'text': 'что'}, {'analysis': [{'lex': 'они', 'gr': 'SPRO,мн=им'}], 'text': 'они'}]


<h3>Токенизация с помощью nltk</h3>

In [5]:
(word_tokenize(text))
# убрала пунктуацию и привела к нижнему регистру
words = [w.lower() for w in word_tokenize(text) if w.isalpha()]

<h3>Парсинг с помощью pymorphy</h3>
<p>Чтобы было удобнее потом сохранить результат в json, сразу перевожу данные в список из словарей</p>

In [6]:
%%time
analist = []

for word in words:
    ana = morph.parse(word)
    first = ana[0] # использую только 1-й вариант разбора

    word = first.word # слово
    tag = first.tag # грам. разбор 
    lemma = first.normal_form # лемма
    prob = first.score # вероятность

    ana_dict = {'text' : word, 'tag' : [tag], 'lemma' : lemma, 'prob' : prob}
    analist.append(ana_dict)

Wall time: 19.6 s


<p>Чтобы можно было вытаскивать теги, не используя встроенные функции OpencorporaTag, добавила их отдельно в словарь, в список в значении "tag".</p>

In [7]:
for parse in analist:
    new_thing = {}
    if parse['tag'][0].POS:
        new_thing.update({'POS':parse['tag'][0].POS})
    if parse['tag'][0].animacy:
        new_thing.update({'animacy':parse['tag'][0].animacy})
    if parse['tag'][0].aspect:
        new_thing.update({'aspect':parse['tag'][0].aspect})
    if parse['tag'][0].case:
        new_thing.update({'case':parse['tag'][0].case})
    if parse['tag'][0].gender:
        new_thing.update({'gender':parse['tag'][0].gender})
    if parse['tag'][0].involvement:
        new_thing.update({'involvement':parse['tag'][0].involvement})
    if parse['tag'][0].mood:
        new_thing.update({'mood':parse['tag'][0].mood})
    if parse['tag'][0].number:
        new_thing.update({'number':parse['tag'][0].number})
    if parse['tag'][0].person:
        new_thing.update({'person':parse['tag'][0].person})
    if parse['tag'][0].tense:
        new_thing.update({'tense':parse['tag'][0].tense})
    if parse['tag'][0].transitivity:
        new_thing.update({'transitivity':parse['tag'][0].transitivity})
    if parse['tag'][0].voice:
        new_thing.update({'voice':parse['tag'][0].voice})
    parse['tag'].append(new_thing)

print(analist[100:105])

[{'text': 'сестре', 'tag': [OpencorporaTag('NOUN,anim,femn sing,datv'), {'POS': 'NOUN', 'animacy': 'anim', 'case': 'datv', 'gender': 'femn', 'number': 'sing'}], 'lemma': 'сестра', 'prob': 0.545454}, {'text': 'было', 'tag': [OpencorporaTag('VERB,impf,intr neut,sing,past,indc'), {'POS': 'VERB', 'aspect': 'impf', 'gender': 'neut', 'mood': 'indc', 'number': 'sing', 'tense': 'past', 'transitivity': 'intr'}], 'lemma': 'быть', 'prob': 0.963576}, {'text': 'ясно', 'tag': [OpencorporaTag('ADVB,Prdx'), {'POS': 'ADVB'}], 'lemma': 'ясно', 'prob': 0.941176}, {'text': 'что', 'tag': [OpencorporaTag('CONJ'), {'POS': 'CONJ'}], 'lemma': 'что', 'prob': 0.791044}, {'text': 'они', 'tag': [OpencorporaTag('NPRO,3per,Anph plur,nomn'), {'POS': 'NPRO', 'case': 'nomn', 'number': 'plur', 'person': '3per'}], 'lemma': 'они', 'prob': 1.0}]


<p>Пришлось оставить исходные значения "tag", так как не все грамматические значения относятся к какому-либо аттрибуту.</p>
<p>Как результат, в значении "tag" список из двух елементов, 1-й - теги в формате OpencorporaTag, 2-й - словарь вида "аттрибут : тег".</p>

<h3>Сохранение в формате json</h3>
<p>Формат json не признает OpencorporaTag, поэтому переделала тег в строку и убрала "OpencorporaTag", оставив только непосредственно теги</p>

In [8]:
with open('igrok_parse.json', 'w', encoding='utf-8') as f:

    for parse in analist:
        parse['tag'][0] = str(parse['tag'][0])
        parse['tag'][0] = re.sub(r'OpencorporaTag()', '', parse['tag'][0])

        json.dump(parse, f)
        f.write('\n')

<p>Проверка, что файл нормально записался, OpencorporaTag исчез.</p>

In [9]:
ana_list = []
with open('igrok_parse.json', encoding='utf-8') as f:
    text = f.read()
    lines = text.splitlines()

for line in lines:
    data = json.loads(line)
    ana_list.append(data)

print(ana_list[100:105])

[{'text': 'сестре', 'tag': ['NOUN,anim,femn sing,datv', {'POS': 'NOUN', 'animacy': 'anim', 'case': 'datv', 'gender': 'femn', 'number': 'sing'}], 'lemma': 'сестра', 'prob': 0.545454}, {'text': 'было', 'tag': ['VERB,impf,intr neut,sing,past,indc', {'POS': 'VERB', 'aspect': 'impf', 'gender': 'neut', 'mood': 'indc', 'number': 'sing', 'tense': 'past', 'transitivity': 'intr'}], 'lemma': 'быть', 'prob': 0.963576}, {'text': 'ясно', 'tag': ['ADVB,Prdx', {'POS': 'ADVB'}], 'lemma': 'ясно', 'prob': 0.941176}, {'text': 'что', 'tag': ['CONJ', {'POS': 'CONJ'}], 'lemma': 'что', 'prob': 0.791044}, {'text': 'они', 'tag': ['NPRO,3per,Anph plur,nomn', {'POS': 'NPRO', 'case': 'nomn', 'number': 'plur', 'person': '3per'}], 'lemma': 'они', 'prob': 1.0}]


<h3>Доля частей речи</h3>

In [10]:
word_count = len(analist) # количество слов в тексте
poslist = [] # список частей речи

# у некоторых слов отсутсвует часть речи (иноязычные слова, римские цифры)
for parse in analist:
    if 'POS' in parse['tag'][1].keys():
        poslist.append(parse['tag'][1]['POS'])

# считает количество вхождений
posdict = Counter(poslist)
# находит и выводит долю
for pos, count in posdict.items():
    print('Доля', pos, str(round(count/word_count * 100, 2))+'%')

Доля NOUN 18.72%
Доля PREP 9.6%
Доля ADJF 9.23%
Доля ADVB 7.61%
Доля NPRO 11.49%
Доля VERB 13.71%
Доля NUMR 1.05%
Доля CONJ 12.47%
Доля PRCL 6.89%
Доля INFN 2.8%
Доля INTJ 0.23%
Доля GRND 0.8%
Доля PRED 0.51%
Доля COMP 0.32%
Доля ADJS 1.03%
Доля PRTS 0.28%
Доля PRTF 0.58%


<h3>Топ-20 глаголов и наречий</h3>

In [11]:
def top_pos(analist,tag):
    lemmalist = [] # список лемм части речи
    top_list = [] # список топ-20

# находит лемму если в разборе соответсвующая часть речи   
    for parse in analist:
        if tag in parse['tag'][0]:
            lemmalist.append(parse['lemma'])
# cчитает вхождения
    lemmadict = Counter(lemmalist)
# находит 20 самых частотных
    for key, value in lemmadict.most_common(20):
        top_list.append(key)

    return top_list
    
    
print('Топ-20 глаголов:',top_pos(analist,'VERB'))
print('Топ-20 наречий:',top_pos(analist,'ADVB'))

Топ-20 глаголов: ['быть', 'мочь', 'знать', 'сказать', 'хотеть', 'говорить', 'стать', 'думать', 'видеть', 'выйти', 'проиграть', 'заметить', 'дать', 'взять', 'любить', 'понимать', 'смотреть', 'отвечать', 'начать', 'выиграть']
Топ-20 наречий: ['теперь', 'вдруг', 'уж', 'уже', 'очень', 'опять', 'здесь', 'тоже', 'тут', 'потому', 'тогда', 'почти', 'тотчас', 'наконец', 'более', 'совсем', 'потом', 'где', 'сейчас', 'действительно']


<h3>N-граммы</h3>

In [12]:
def find_ngrams(text, number):
# делает список всех н-грамм
    ngram_list = list(ngrams(text, number))
# создает частотный словарь
    ngrams_rank = Counter(ngram_list)
# находит 25 самых частотных
    top_20 = ngrams_rank.most_common(25)
# убирает все лишние скобки
    for ngram in top_20:
        n_words = ' '.join(ngram[0])
        frequency = ngram[1]
        
        print(n_words, frequency)

<p><b>Биграммы</b></p>

In [13]:
find_ngrams(words, 2)

что я 116
mademoiselle blanche 96
и не 89
может быть 75
я не 73
мистер астлей 69
всё это 55
и в 54
потому что 51
на меня 47
и я 47
что вы 47
у меня 45
что она 44
тотчас же 42
ничего не 42
не знаю 41
и даже 41
ко мне 40
алексей иванович 40
что он 38
со мною 36
я и 36
да и 36
и с 33


<p><b>Триграммы</b></p>

In [14]:
find_ngrams(words, 3)

по крайней мере 27
в самом деле 18
до сих пор 18
пятьдесят тысяч франков 15
к тому же 12
даже и не 12
и в самом 11
может быть и 10
о том что 10
на этот раз 10
что я не 10
я ничего не 10
во всяком случае 9
и mademoiselle blanche 9
что же касается 9
же касается до 9
и уж конечно 8
я так и 8
mademoiselle blanche и 8
а между тем 8
в том что 8
я не знаю 8
в эту минуту 8
я не могу 7
для того чтобы 7
